In [103]:
import pandas as pd                                     
import numpy as np                                      
import os 



import matplotlib.pyplot as plt

from datetime import datetime

%matplotlib inline
import seaborn as sns                                   # For pretty plots
import gensim


from os import path
from wordcloud import WordCloud
from wordcloud import STOPWORDS
from PIL import Image
from gensim import corpora, models, similarities

In [104]:
d = path.dirname("hillary-clinton-emails/")

# Read the whole text.
#emailJoined = open(path.join(d, 'Emails.csv')).read(200000)
emailJoined = open(path.join(d, 'Emails.csv')).read()



[From Strings to vector](https://radimrehurek.com/gensim/tut1.html)

In [105]:
#stoplist = set('for a of the and to in - 1 2 3 4 5 6 7 8 9 a b c d e f g h i j k l m n o p q r s t u v w x y z é + " * ç % & / ()'.split())
#print(stoplist)
#documents = ["Human machine interface for lab abc computer applications"]
#type(documents.split())

documents = emailJoined.replace('.',',').replace('<',',').replace('>',',').replace(' ',',').replace(':',',').split(',')
#documents
                               


In [124]:
stoplist = set(STOPWORDS)

for i in range(1,100) :
    stoplist.add(i)

stoplist.add("no.")
stoplist.add("no")
stoplist.add("pm")
stoplist.add("am")
stoplist.add("doc")
stoplist.add("docx")
stoplist.add("pdf")
stoplist.add("re")
stoplist.add("fw")


#print(stoplist)


In [125]:
texts = [[word for word in document.lower().split() if word not in stoplist and len(word) > 1]
         for document in documents]

#print(texts)
#texts

In [126]:
# remove words that appear only once
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
     for token in text:
            frequency[token] += 1

texts = [[token for token in text if frequency[token] > 10]
          for text in texts]



For representing the questions only by ids (intergers) we need a dictonary to store the mapping.

In [127]:
dictionary = corpora.Dictionary(texts)
dictionary.save("part3/deerwester.dict")


In [128]:
print(dictionary)

Dictionary(15489 unique tokens: ['stupid', 'bipartisan', 'loretta', 'vietnamese', 'proper']...)


In [129]:
#print(dictionary.token2id)

--------------------

In [130]:
corpus = [dictionary.doc2bow(text) for text in texts]

#splited = emailJoined.split(',')
#corpus = [dictionary.doc2bow(text) for text in splited]




corpora.MmCorpus.serialize("part3/deerwester.mm", corpus)  # store to disk, for later use
#print(corpus)


--------------------

[Transformation interface](https://radimrehurek.com/gensim/tut2.html#available-transformations)

In [131]:
dictionary = corpora.Dictionary.load("part3/deerwester.dict")
corpus = corpora.MmCorpus('part3/deerwester.mm')

In [132]:
#tfidf = models.TfidfModel(corpus) # step 1 -- initialize a model

In [133]:
model = models.LdaModel(corpus, id2word=dictionary, num_topics=5)

In [134]:
print(model)

LdaModel(num_terms=15489, num_topics=5, decay=0.5, chunksize=2000)


In [135]:
model.print_topics(30)

[(0,
  '0.082*"08/31/2015" + 0.064*"date" + 0.045*"subject" + 0.019*"message" + 0.014*"huma" + 0.010*"one" + 0.010*"secretary" + 0.009*"time" + 0.009*"security" + 0.008*"states"'),
 (1,
  '0.017*"new" + 0.016*"com>" + 0.014*"government" + 0.013*"<hrod17@clintonemail" + 0.012*"gov\'" + 0.011*"dec" + 0.011*"also" + 0.011*"gov>"" + 0.010*"see" + 0.010*"pm""'),
 (2,
  '0.047*"sent" + 0.040*"release" + 0.028*"will" + 0.026*"""" + 0.023*"part" + 0.020*"gov>" + 0.019*""unclassified" + 0.017*"said" + 0.015*"original" + 0.013*"sullivan"'),
 (3,
  '0.093*"f-2014-20439" + 0.080*"00" + 0.068*"state" + 0.062*"department" + 0.040*"unclassified" + 0.037*"00+00" + 0.027*"december" + 0.020*"cheryl" + 0.019*"full" + 0.013*"jacob"'),
 (4,
  '0.068*"2010" + 0.062*"case" + 0.020*"clinton_email_august_release" + 0.020*"2015-08-31t04" + 0.016*"b6" + 0.013*"10" + 0.010*"15" + 0.009*"people" + 0.008*"start" + 0.007*"14"')]

In [122]:
oldModel = model

In [123]:
oldModel.print_topics(30)

[(0,
  '0.021*"re" + 0.020*"2015-08-31t04" + 0.019*"message" + 0.016*"b6" + 0.015*"com>" + 0.014*"jacob" + 0.012*"<hrod17@clintonemail" + 0.012*"may" + 0.012*"cc" + 0.011*"11"'),
 (1,
  '0.095*"f-2014-20439" + 0.066*"2010" + 0.041*"unclassified" + 0.022*"part" + 0.018*""unclassified" + 0.017*"fw" + 0.016*"said" + 0.014*"original" + 0.013*"10" + 0.012*"government"'),
 (2,
  '0.061*"date" + 0.046*"sent" + 0.039*"release" + 0.028*"december" + 0.020*"08/31/2015" + 0.019*"gov>" + 0.016*"new" + 0.010*"gov>"" + 0.010*"call" + 0.009*"president"'),
 (3,
  '0.085*"00" + 0.044*"subject" + 0.039*"00+00" + 0.027*"will" + 0.026*"""" + 0.021*"cheryl" + 0.013*"huma" + 0.013*"mills" + 0.010*"15" + 0.010*"american"'),
 (4,
  '0.074*"state" + 0.068*"department" + 0.064*"case" + 0.060*"08/31/2015" + 0.021*"full" + 0.020*"clinton_email_august_release" + 0.011*"dec" + 0.010*"pm"" + 0.008*"foreign" + 0.008*"united"')]